# Configuration

This notebook generates the table for differences by configuration.

In [1]:
%reload_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
from typing import Dict, Tuple
from depsurf import DepKind, DiffResult, IssueEnum, VersionGroup
from utils import (
    load_pkl,
    GRAY_DASH,
    save_latex,
    FLAVOR_NAMES,
    ARCH_NAMES,
    texttt,
    footnotesize,
    mini_bar,
)

data: DiffResult = load_pkl("config")

ISSUES = {
    IssueEnum.NEW: "#",
    IssueEnum.ADD: "$+$",
    IssueEnum.REMOVE: "$-$",
    IssueEnum.CHANGE: r"$\Delta$",
}

KINDS = {
    DepKind.STRUCT: "Struct",
    DepKind.CONFIG: "Config",
    DepKind.FUNC: "Func",
    DepKind.TRACEPOINT: "Tracept",
    # DepKind.TRACEPOINT: r"\shortstack{Tracept\\($\Delta=0$)}",
    DepKind.SYSCALL: r"\shortstack{Native\\Syscall}",
    DepKind.REGISTER: "Register",
}

ROWS = [
    (DepKind.CONFIG, IssueEnum.NEW),
    *(
        (kind, issue)
        for kind in (DepKind.FUNC, DepKind.STRUCT, DepKind.TRACEPOINT)
        for issue in ISSUES
    ),
    *(
        (DepKind.SYSCALL, issue)
        for issue in [i for i in ISSUES if i != IssueEnum.CHANGE]
    ),
    (DepKind.REGISTER, IssueEnum.CHANGE),
]

GROUPS = {
    VersionGroup.ARCH: "Architecture",
    VersionGroup.FLAVOR: "Flavor",
}

DEFAULT = (r"\multicolumn{1}{c|}{def-}", r"\multicolumn{1}{c|}{ault}")

COLORS = {
    DepKind.FUNC: "func",
    DepKind.STRUCT: "struct",
    DepKind.TRACEPOINT: "tp",
    DepKind.SYSCALL: "syscall",
}


def format_val(v) -> str:
    if v > 1000:
        return f"{v / 1000:.1f}k"
    if v == 0:
        return GRAY_DASH
    return str(v)


table: Dict[Tuple[str, str], Dict[Tuple[DepKind, IssueEnum], str]] = {}

def_pair_result = list(data.group_results[VersionGroup.ARCH].pair_results.values())[0]
col = {(kind, issue): GRAY_DASH for kind, issue in ROWS}
for kind, kind_result in def_pair_result.iter_kinds():
    for issue, count in kind_result.iter_issues():
        if issue == IssueEnum.OLD:
            col[(kind, IssueEnum.NEW)] = format_val(count)
            continue
table[DEFAULT] = col

max_counts = {
    kind: max(
        count
        for group, group_result in data.iter_groups()
        for pair, pair_result in group_result.iter_pairs()
        for issue, count in pair_result.kind_results[kind].iter_issues()
        if issue not in (IssueEnum.OLD, IssueEnum.NEW)
    )
    for kind in [DepKind.FUNC, DepKind.STRUCT, DepKind.TRACEPOINT, DepKind.SYSCALL]
}

for group, group_result in data.iter_groups():
    if group not in GROUPS:
        continue
    for pair, pair_result in group_result.iter_pairs():
        col = {(kind, issue): GRAY_DASH for kind, issue in ROWS}
        for kind, kind_result in pair_result.iter_kinds():
            for issue, count in kind_result.iter_issues():
                if (kind, issue) not in ROWS:
                    continue
                text = format_val(count)
                if issue != IssueEnum.NEW:
                    text = mini_bar(
                        text,
                        count / max_counts[kind],
                        0.8,
                        COLORS[kind],
                    )
                col[(kind, issue)] = text
        if group == VersionGroup.ARCH:
            name = texttt(footnotesize(ARCH_NAMES[pair.v2.arch]))
            col[(DepKind.REGISTER, IssueEnum.CHANGE)] = "Yes"
        elif group == VersionGroup.FLAVOR:
            name = FLAVOR_NAMES[pair.v2.flavor]
        table[(GROUPS[group], name)] = col


df = pd.DataFrame(
    {
        (group, name): {
            (KINDS[kind], ISSUES[issue]): text for (kind, issue), text in col.items()
        }
        for (group, name), col in table.items()
    }
)

latex = df.to_latex(
    multicolumn_format="c|", column_format=r"p{7ex}p{1.5ex}|r|rrrr|rrrr", multirow=True
)

save_latex(latex, "cfg", rotate=False)
df

[ utils_pickle.py:18 ] INFO: Loding config from /Users/szhong/Code/DepSurf/output/config.pkl
[  utils_latex.py:118] INFO: Saved cfg to /Users/szhong/Code/DepSurf/paper/tabs/cfg.tex


\multicolumn{1}{c|}{def-}                                                Architecture                                                                                                                                                                                                                                               Flavor                                                                                                                                                                                                   
                                      \multicolumn{1}{c|}{ault}                               \texttt{\footnotesize{arm64}}                               \texttt{\footnotesize{arm32}}                                 \texttt{\footnotesize{ppc}}                               \texttt{\footnotesize{riscv}}                                                              AWS                                                            Azure                                                              GCP                                                          low-lat
Config                       #                             8.8k                                                        9.6k                                                        9.6k                                                        8.1k                                                        7.6k                                                             6.4k                                                             5.3k                                                             8.6k                                                             8.8k
Func                         #                            48.0k                                                       49.4k                                                       48.8k                                                       42.8k                                                       36.6k                                                            46.5k                                                            45.5k                                                            48.1k                                                            48.0k
                             $+$           \color{lightgray}{-}                \colorbox{func}{\makebox[{0.55cm}][r]{9.2k}}               \colorbox{func}{\makebox[{0.75cm}][r]{12.6k}}                \colorbox{func}{\makebox[{0.32cm}][r]{5.4k}}                \colorbox{func}{\makebox[{0.12cm}][r]{2.1k}}                      \colorbox{func}{\makebox[{0.02cm}][r]{328}}                      \colorbox{func}{\makebox[{0.06cm}][r]{992}}                      \colorbox{func}{\makebox[{0.03cm}][r]{450}}                       \colorbox{func}{\makebox[{0.00cm}][r]{57}}
                             $-$           \color{lightgray}{-}                \colorbox{func}{\makebox[{0.47cm}][r]{7.9k}}               \colorbox{func}{\makebox[{0.70cm}][r]{11.8k}}               \colorbox{func}{\makebox[{0.63cm}][r]{10.6k}}               \colorbox{func}{\makebox[{0.80cm}][r]{13.5k}}                     \colorbox{func}{\makebox[{0.11cm}][r]{1.8k}}                     \colorbox{func}{\makebox[{0.21cm}][r]{3.5k}}                      \colorbox{func}{\makebox[{0.02cm}][r]{319}}                       \colorbox{func}{\makebox[{0.00cm}][r]{41}}
                             $\Delta$      \color{lightgray}{-}                 \colorbox{func}{\makebox[{0.01cm}][r]{120}}                 \colorbox{func}{\makebox[{0.01cm}][r]{106}}                 \colorbox{func}{\makebox[{0.01cm}][r]{137}}                 \colorbox{func}{\makebox[{0.01cm}][r]{101}}                        \colorbox{func}{\makebox[{0.00cm}][r]{2}}                       \colorbox{func}{\makebox[{0.00cm}][r]{10}}                        \colorbox{func}{\makebox[{0.00cm}][r]{1}}     \colorbox{func}{\makebox[{0.00cm}][r]{\color{lightgray}{-}}}
Struct                       #                             8.4k                           